In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    data_files = np.array(data['filenames'])
    data_targets = data['target']
    return data_files, data_targets


train_files, train_targets = load_dataset('data/')


# load list of dog names
names = [item[5:-1] for item in sorted(glob("data/*/"))]

# print statistics about the dataset
print('There are %d total categories.' % len(names))
print('There are %d total images.' % len(train_files))

Using TensorFlow backend.


There are 8 total categories.
There are 486 total images.


In [2]:
sorted(names)

['fifty',
 'five',
 'five hundred',
 'one hundred',
 'one thousand',
 'ten',
 'twenty',
 'two hundred']

In [5]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from tqdm import tqdm
from PIL import ImageFile    


def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (299, 299, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 299, 299, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [17]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
# pre-process the train data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')

100%|██████████| 486/486 [00:06<00:00, 80.11it/s]


In [18]:
train_tensors[0].shape

(299, 299, 3)

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog


class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """

    def __init__(self):
        pass

    def fit(self, X, y=None):
        """returns itself"""
        return self

    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])


class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """

    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)

        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [36]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import skimage

# create an instance of each transformer
grayify = RGB2GrayTransformer()
hogify = HogTransformer(
    pixels_per_cell=(8, 8),
    cells_per_block=(2,2),
    orientations=9,
    block_norm='L2-Hys'
)
scalify = StandardScaler()

# call fit_transform on each transform converting X_train step by step
X_train_gray = grayify.fit_transform(train_tensors)
X_train_hog = hogify.fit_transform(X_train_gray)
scalify.fit(X_train_hog)
X_train_prepared = scalify.transform(X_train_hog)

In [37]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [38]:
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=5)

model = SVC()
model.fit(X_train_prepared, train_targets)
# Nested CV with parameter optimization
nested_lin_1 = cross_val_score(model, X=X_train_prepared, y=train_targets,
                               scoring='accuracy', cv=outer_cv)
print(nested_lin_1.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default 

0.9446193523114476


In [40]:
nested_lin_1

array([0.96226415, 0.94230769, 0.92307692, 0.93877551, 0.97916667,
       0.89583333, 0.93617021, 0.95652174, 0.95652174, 0.95555556])

In [41]:
from sklearn.externals import joblib
filename = 'finalized_model4.sav'
joblib.dump(model, filename)

['finalized_model4.sav']

In [45]:
filename = 'Scaler1.sav'
joblib.dump(scalify, filename)

['Scaler1.sav']